<a href="https://colab.research.google.com/github/ealbasiri/English_text_normalization/blob/main/EnglishCardinals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
### WARNING: This notebook will not work in a Colab environment. 

BRANCH= 'main'

!git clone -b $BRANCH https://github.com/NVIDIA/NeMo
%cd NeMo
!./reinstall.sh

Cloning into 'NeMo'...
remote: Enumerating objects: 99462, done.
remote: Counting objects: 100% (6450/6450), done.
remote: Compressing objects: 100% (2278/2278), done.
remote: Total 99462 (delta 4770), reused 5362 (delta 4146), pack-reused 93012
Receiving objects: 100% (99462/99462), 215.51 MiB | 26.17 MiB/s, done.
Resolving deltas: 100% (74351/74351), done.
/content/NeMo
Uninstalling stuff
     |████████████████████████████████| 1.1 MB 5.0 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Installing nemo an

In [4]:
import pynini
import nemo_text_processing

from pynini.lib import pynutil

In [5]:
from nemo_text_processing.text_normalization.en.graph_utils import GraphFst, NEMO_DIGIT, delete_space, NEMO_SIGMA, NEMO_NOT_QUOTE, delete_extra_space, NEMO_NON_BREAKING_SPACE
from nemo_text_processing.text_normalization.normalize import Normalizer

from nemo_text_processing.inverse_text_normalization.fr.taggers.cardinal import CardinalFst
from nemo_text_processing.inverse_text_normalization.fr.taggers.decimal import DecimalFst
from nemo_text_processing.inverse_text_normalization.fr.taggers.money import MoneyFst
from nemo_text_processing.inverse_text_normalization.fr.taggers.ordinal import OrdinalFst
from nemo_text_processing.inverse_text_normalization.fr.taggers.punctuation import PunctuationFst
from nemo_text_processing.inverse_text_normalization.fr.taggers.time import TimeFst
from nemo_text_processing.inverse_text_normalization.fr.taggers.whitelist import WhiteListFst
from nemo_text_processing.inverse_text_normalization.fr.taggers.word import WordFst
from nemo_text_processing.inverse_text_normalization.fr.verbalizers.cardinal import CardinalFst
from nemo_text_processing.inverse_text_normalization.fr.verbalizers.decimal import DecimalFst
from nemo_text_processing.inverse_text_normalization.fr.verbalizers.money import MoneyFst
from nemo_text_processing.inverse_text_normalization.fr.verbalizers.ordinal import OrdinalFst
from nemo_text_processing.inverse_text_normalization.fr.verbalizers.time import TimeFst
from nemo_text_processing.inverse_text_normalization.fr.verbalizers.whitelist import WhiteListFst
from nemo_text_processing.inverse_text_normalization.fr.verbalizers.word import WordFst

[NeMo W 2022-04-03 21:48:00 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-04-03 21:48:01 experimental:28] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [6]:
LANGUAGE= "MY_LANGUAGE" # Change this to your desired language

In [7]:
%cd nemo_text_processing/inverse_text_normalization/
!mkdir {LANGUAGE}
!mkdir "{LANGUAGE}/taggers"
!mkdir "{LANGUAGE}/verbalizers"
!mkdir "{LANGUAGE}/data"
%cd {LANGUAGE}
!pwd && ls

/content/NeMo/nemo_text_processing/inverse_text_normalization
/content/NeMo/nemo_text_processing/inverse_text_normalization/MY_LANGUAGE
/content/NeMo/nemo_text_processing/inverse_text_normalization/MY_LANGUAGE
data  taggers  verbalizers


In [8]:
!cp ../../text_normalization/en/graph_utils.py .
!cp ../../text_normalization/en/utils.py .
! cd ../../..

In [9]:
from pynini.lib import pynutil

def apply_fst(text, fst):
  """ Given a string input, returns the output string
  produced by traversing the path with lowest weight.
  If no valid path accepts input string, returns an
  error.
  """
  try:
     print(pynini.shortestpath(text @ fst).string())
  except pynini.FstOpError:
    print(f"Error: No valid output with given input: '{text}'")

In [27]:
zeroE= pynini.string_map([("zero", "0")])
digitsE= pynini.string_map([("one", "1"),("two", "2"),("three", "3"),("four", "4"), ("five", "5"), ("six", "6"), 
                          ("seven", "7"), ("eight", "8"), ("nine", "9")])

In [28]:

teensE= pynini.string_map ([("eleven", "11"),
                            ("twelve", "12"),
                            ("thirteen", "13"),
                            ("fourteen", "14"),
                            ("fifteen", "15"),
                            ("sixteen", "16"),
                            ("seventeen", "17"),
                            ("eighteen", "18"), 
                            ("nineteen", "19"),
                            ])

In [29]:
tensE= pynini.string_map([("twenty", "2"),
                            ("thirty", "3"), 
                            ("fourty", "4"), 
                            ("sixty", "6"), 
                            ("seventy", "7"), 
                            ("eighty", "8"), 
                            ("ninety", "9"),
                            ("hundred", "10")])

In [30]:
graph_digitsE = digitsE | pynutil.insert("0")
delete_space = pynutil.delete(pynini.union("-", " "))
graph_tensE = tensE + graph_digitsE | tensE + delete_space + graph_digitsE
graph_allE = zeroE | graph_digitsE | teensE | graph_tensE 

In [31]:
apply_fst("two", graph_allE)
apply_fst("thirty-three", graph_allE)
apply_fst("sixty", graph_allE)
apply_fst("eighteen", graph_allE)
apply_fst("sixteen", graph_allE)
apply_fst("twelve", graph_allE)
apply_fst("one", graph_allE)
apply_fst("twenty", graph_allE)
apply_fst("twenty-one", graph_allE)
apply_fst("hundred", graph_allE)
apply_fst("zero", graph_allE)
apply_fst("ninety-eight", graph_allE)
apply_fst("sixty nine", graph_allE)

2
33
60
18
16
12
1
20
21
100
0
98
69


In [68]:
zeroE= pynini.string_map([("0", "zero")])
digitsE= pynini.string_map( [("1", "one"),("2", "two"),("3", "three"),("4", "four"), ("5", "five"), ("6", "six"), 
                          ("7", "seven"), ("8", "eight"), ("9", "nine") ])

In [74]:
teensE= pynini.string_map ([("11", "eleven"),
                            ("12", "twelve"),
                            ("13", "thirteen"),
                            ("14", "fourteen"),
                            ("15", "fifteen"),
                            ("16", "sixteen"),
                            ("17", "seventeen"),
                            ("18", "eighteen"), 
                            ("19", "nineteen"),
                            ])

In [75]:
tens_plusE= pynini.string_map([("2", "twenty"),
                            ("3", "thirty"), 
                            ("4", "fourty"), 
                            ("6", "sixty"), 
                            ("7", "seventy"), 
                            ("8", "eighty"), 
                            ("9", "ninety")])

In [79]:
tensE= pynini.string_map([("20", "twenty"),
                            ("30", "thirty"), 
                            ("40", "fourty"), 
                            ("60", "sixty"), 
                            ("70", "seventy"), 
                            ("80", "eighty"), 
                            ("90", "ninety"),
                          ("100", "hundred")])

In [80]:
graph_digitsE = digitsE
insert_space = pynutil.insert("-")
graph_tens_plusE = tens_plusE + insert_space + graph_digitsE
graph_allE = zeroE | graph_digitsE | teensE | tensE |graph_tens_plusE  

In [81]:
apply_fst("2", graph_allE)
apply_fst("33", graph_allE)
apply_fst("60", graph_allE)
apply_fst("18", graph_allE)
apply_fst("16", graph_allE)
apply_fst("12", graph_allE)
apply_fst("1", graph_allE)
apply_fst("20", graph_allE)
apply_fst("21", graph_allE)
apply_fst("100", graph_allE)
apply_fst("0", graph_allE)

two
thirty-three
sixty
eighteen
sixteen
twelve
one
twenty
twenty-one
hundred
zero
